In [ ]:
!pip install -q yt-dlp openai-whisper

import os
import subprocess
import whisper
from IPython.display import Audio, display
from google.colab import files
import torch;

print('You are ready to go!')
print(f"FYI: Cuda Availability: {torch.cuda.is_available()}")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s

In [ ]:
def download_youtube_audio(url, format='mp3'):
    """YouTube URLから音声をダウンロードする関数 (yt-dlp を使用)"""
    if not url:
        print('URLが入力されていません。')
        return None

    try:
        # Colab 上で使う作業ディレクトリ例（お好みで変更可）
        workspace_dir = '/content/whisper_workspace'
        os.makedirs(workspace_dir, exist_ok=True)
        os.chdir(workspace_dir)

        # 動画IDを抽出して出力テンプレートを作成
        # 例: youtube_XXXXXXXXXXX.mp3
        video_id = url.split('watch?v=')[-1][:11]
        output_template = f'youtube_{video_id}.%(ext)s'

        # yt-dlp コマンド（指定のオプションを含む）
        command = (
            f"yt-dlp "
            f"--hls-use-mpegts "
            f"--force-overwrites "
            f"-x --audio-format {format} "
            f"-o '{output_template}' "
            f"'{url}'"
        )
        print('音声ダウンロード中...')
        subprocess.run(command, shell=True, check=True)

        # ダウンロードされたファイルが想定通り存在するか確認
        expected_file = f'youtube_{video_id}.{format}'
        if os.path.exists(expected_file):
            abs_path = os.path.abspath(expected_file)
            print(f'ダウンロード完了: {abs_path}')
            return abs_path
        else:
            print('音声ファイルが見つかりませんでした。')
            return None

    except Exception as e:
        print(f'エラーが発生しました: {str(e)}')
        return None
def generate_srt(
    audio_file: str,
    model_size: str = "turbo",
    language: str = "auto"
) -> str:
    """
    openai-whisper を使って音声ファイル (mp3 など) から文字起こしをし、
    SRT形式の字幕ファイルを生成してパスを返す。
    """

    if not os.path.exists(audio_file):
        print(f"音声ファイルが見つかりません: {audio_file}")
        return None

    try:
        # Whisper モデルをロード (初回実行時はモデルの自動ダウンロードが入る)
        print(f"Whisper モデル '{model_size}' をロードしています...")
        model = whisper.load_model(model_size)

        # language='auto' としたい場合は引数を付けないなどで切り替え
        if language.lower() == "auto":
            result = model.transcribe(audio_file)
        else:
            result = model.transcribe(audio_file, language=language)

        # SRT ファイル名は元ファイル名の拡張子だけ差し替える例
        base_name = os.path.splitext(audio_file)[0]
        srt_path = f"{base_name}.srt"

        # 結果を SRT 形式で書き出し
        write_srt(result["segments"], srt_path)
        print(f"SRTファイルを生成しました: {srt_path}")

        return srt_path

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return

def write_srt(segments, srt_path: str):
    """
    Whisper の result["segments"] をベースに、
    シンプルに SRT 形式のファイルを生成するヘルパー関数。
    """
    def sec_to_timestamp(sec: float):
        hours = int(sec // 3600)
        minutes = int((sec % 3600) // 60)
        seconds = int(sec % 60)
        milliseconds = int((sec - int(sec)) * 1000)
        return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

    with open(srt_path, "w", encoding="utf-8") as f:
        for i, seg in enumerate(segments, start=1):
            start_time = sec_to_timestamp(seg["start"])
            end_time = sec_to_timestamp(seg["end"])
            text = seg["text"].strip()
            f.write(f"{i}\n")
            f.write(f"{start_time} --> {end_time}\n")
            f.write(f"{text}\n\n")

def process_youtube_to_srt(url, audio_format='mp3', model_size='turbo', language='auto' ):
    """
    1. 指定URL から音声をダウンロード
    2. Whisper で字幕を生成 (SRT)
    """
    # 1. ダウンロード（こちらが先ほど定義した関数）
    audio_path = download_youtube_audio(url, format=audio_format)
    if not audio_path:
        print("音声ダウンロードに失敗しました。終了します。")
        return None

    # 2. Whisper で字幕生成
    print("字幕生成処理開始しました。...")

    srt_path = generate_srt(audio_path, model_size=model_size, language=language)

    if not srt_path:
        print("字幕生成に失敗しました。")
        return None

    return srt_path



In [ ]:

# 実行例 (対話的)
url = input('YouTube動画のURLを入力してください: ').strip()
lang_input = input('言語コード (選択肢: ja, en, auto / デフォルト: auto): ').strip()
language = lang_input if lang_input else 'auto'
model_size_input = input('使用モデルサイズ (選択肢: tiny, base, small, medium, large, turbo / デフォルト: turbo): ').strip()
model_size = model_size_input if model_size_input else 'turbo'

srt_file = process_youtube_to_srt(
    url=url,
    audio_format='mp3',
    model_size=model_size,   # Whisperのモデルサイズ
    language=language,
)

# SRTファイルのDLリンク表示処理:
if srt_file:
    answer = input("お待たせしました！ 生成されたファイルをダウンロードしますか？ (y/n): ")
    if answer.lower() == 'y':
        files.download(srt_file)
        print("Thank you for using! ")
    else:
        print("必要であれば、'/whisper_workspace' の下にファイルは保存されています。")

YouTube動画のURLを入力してください: https://www.youtube.com/watch?v=KN0Cgt06ev4
言語コード (選択肢: ja, en, auto / デフォルト: auto): 
使用モデルサイズ (選択肢: tiny, base, small, medium, large, turbo / デフォルト: turbo): large
音声ダウンロード中...
ダウンロード完了: /content/whisper_workspace/youtube_KN0Cgt06ev4.mp3
字幕生成処理開始しました。...
Whisper モデル 'large' をロードしています...


100%|██████████████████████████████████████| 2.88G/2.88G [00:28<00:00, 109MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

SRTファイルを生成しました: /content/whisper_workspace/youtube_KN0Cgt06ev4.srt
お待たせしました！ 生成されたファイルをダウンロードしますか？ (y/n): y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Thank you for using! 
